In [1]:
import nltk
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.compose import ColumnTransformer
from sklearn.experimental import enable_iterative_imputer
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.preprocessing import StandardScaler, RobustScaler, MinMaxScaler
from sklearn.feature_selection import RFE, RFECV, f_regression, chi2
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import GridSearchCV, KFold, RandomizedSearchCV, HalvingGridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.feature_selection import SelectKBest
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, recall_score, precision_score, f1_score, roc_auc_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB, BernoulliNB, MultinomialNB
from sklearn.preprocessing import PowerTransformer, QuantileTransformer
from mlxtend.feature_selection import SequentialFeatureSelector as SFS
from sklearn.neighbors import  KNeighborsClassifier
from datetime import datetime as dt
from sklearn.impute import SimpleImputer, KNNImputer, IterativeImputer
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import GradientBoostingClassifier, AdaBoostClassifier, BaggingClassifier, StackingClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, LabelEncoder
from imblearn.combine import SMOTETomek
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer
from mlxtend.feature_selection import ColumnSelector

# explicitly require this experimental feature
from sklearn.experimental import enable_halving_search_cv # noqa
# now you can import normally from model_selection
from sklearn.model_selection import HalvingGridSearchCV

import warnings
# Settings the warnings to be ignored
warnings.filterwarnings('ignore')

In [2]:
# Displaying all columns
pd.set_option('display.max_columns', None)

#Display all data in columns
pd.set_option("display.max_colwidth", None)

np.set_printoptions(threshold=None, edgeitems=None, linewidth=None, suppress=None)
pd.set_option("display.max_rows", None)
pd.set_option("display.expand_frame_repr", True)
pd.set_option('display.width', 1000)


In [3]:
train_df = pd.read_csv('train.csv')

In [4]:
train_df.shape

(120000, 3)

In [5]:
train_df.head(5)

,Class Index,Title,Description
0,3,Wall St. Bears Claw Back Into the Black (Reuters),"Reuters - Short-sellers, Wall Street's dwindling\band of ultra-cynics, are seeing green again."
1,3,Carlyle Looks Toward Commercial Aerospace (Reuters),"Reuters - Private investment firm Carlyle Group,\which has a reputation for making well-timed and occasionally\controversial plays in the defense industry, has quietly placed\its bets on another part of the market."
2,3,Oil and Economy Cloud Stocks' Outlook (Reuters),Reuters - Soaring crude prices plus worries\about the economy and the outlook for earnings are expected to\hang over the stock market next week during the depth of the\summer doldrums.
3,3,Iraq Halts Oil Exports from Main Southern Pipeline (Reuters),"Reuters - Authorities have halted oil export\flows from the main pipeline in southern Iraq after\intelligence showed a rebel militia could strike\infrastructure, an oil official said on Saturday."
4,3,"Oil prices soar to all-time record, posing new menace to US economy (AFP)","AFP - Tearaway world oil prices, toppling records and straining wallets, present a new economic menace barely three months before the US presidential elections."


In [15]:
train_df['Description_preprocessed'] = train_df['Description']

In [6]:
#Dependencies
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /home/mist/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/mist/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/mist/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [7]:
train_df.duplicated().sum()

0

In [8]:
train_df.isnull().sum()

Class Index    0
Title          0
Description    0
dtype: int64

In [9]:
train_df['Class Index'].value_counts()  #Not imbalanced

Class Index
3    30000
4    30000
2    30000
1    30000
Name: count, dtype: int64

In [10]:
#Preprocessing the description

In [11]:
import spacy

nlp = spacy.load('en_core_web_sm')

2023-08-23 23:41:58.256033: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-08-23 23:41:58.658600: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-08-23 23:42:00.866451: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-08-23 23:42:00.894553: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-23 23:42:06.643931: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Co

In [12]:
!pip install contractions
!pip install wordcloud
!pip install autocorrect
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /home/mist/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/mist/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [18]:
#PreprocesDescriptionng 'overview' column

#Lowercasing

train_df['Description_preprocessed'] = train_df['Description_preprocessed'].str.lower()


#Removing Contradictions

import contractions

def remove_contradictions(text):

    return " ".join([contractions.fix(word.text) for word in nlp(text)])

train_df['Description_preprocessed']= train_df['Description_preprocessed'].apply(remove_contradictions)



# Removing HTML tags

import re

def remove_html(text):
    pattern = re.compile('<.*?>')
    return pattern.sub(r'', text)

train_df['Description_preprocessed'] = train_df['Description_preprocessed'].apply(remove_html)


#Removing URL

import re

def remove_url(text):
    pattern = re.compile(r'https?://\S+|www\.\S+')
    return pattern.sub(r'', text)

train_df['Description_preprocessed']= train_df['Description_preprocessed'].apply(remove_url)



def remove_at_the_rate_and_mentions(text):

      return ' '.join(re.sub("(#[A-Za-z0-9]+)|(@[A-Za-z0-9]+)"," ",text).split())


train_df['Description_preprocessed']= train_df['Description_preprocessed'].apply(remove_at_the_rate_and_mentions)



#Remmove punctuation

import string

punc = string.punctuation

def  remove_punc(text):

    return text.translate(str.maketrans('', '', punc))

train_df['Description_preprocessed']= train_df['Description_preprocessed'].apply(remove_punc)




# Removing stop words


from nltk.corpus import stopwords

stopwords = stopwords.words('english')

def remove_stop_words(text):
    ls = []
    new = []

    ls = nlp(text)

    for word in ls:
        if word.text not in stopwords:

            new.append(word.text)

    return ' '.join(new)

train_df['Description_preprocessed'] = train_df['Description_preprocessed'].apply(remove_stop_words)



def Lemmetization(text):

    return " ".join([word.lemma_ for word in nlp(text)])

train_df['Description_preprocessed'] = train_df['Description_preprocessed'].apply(Lemmetization)




In [19]:
#Preprocesing the 'Title' column

In [21]:
train_df['Title_processed'] = train_df['Title']

In [22]:
#Preprocessing 'overview' column

#Lowercasing

train_df['Title_processed'] = train_df['Title_processed'].str.lower()


#Removing Contradictions

import contractions

def remove_contradictions(text):

    return " ".join([contractions.fix(word.text) for word in nlp(text)])

train_df['Title_processed']= train_df['Title_processed'].apply(remove_contradictions)




# Removing HTML tags

import re

def remove_html(text):
    pattern = re.compile('<.*?>')
    return pattern.sub(r'', text)

train_df['Title_processed'] = train_df['Title_processed'].apply(remove_html)



#Removing URL

import re

def remove_url(text):
    pattern = re.compile(r'https?://\S+|www\.\S+')
    return pattern.sub(r'', text)

train_df['Title_processed']= train_df['Title_processed'].apply(remove_url)




def remove_at_the_rate_and_mentions(text):

      return ' '.join(re.sub("(#[A-Za-z0-9]+)|(@[A-Za-z0-9]+)"," ",text).split())


train_df['Title_processed']= train_df['Title_processed'].apply(remove_at_the_rate_and_mentions)


#Remmove punctuation

import string

punc = string.punctuation

def  remove_punc(text):

    return text.translate(str.maketrans('', '', punc))

train_df['Title_processed']= train_df['Title_processed'].apply(remove_punc)




from nltk.corpus import stopwords

stopwords = stopwords.words('english')

def remove_stop_words(text):
    ls = []
    new = []

    ls = nlp(text)

    for word in ls:
        if word.text not in stopwords:

            new.append(word.text)

    return ' '.join(new)

train_df['Title_processed'] = train_df['Title_processed'].apply(remove_stop_words)



def Lemmetization(text):

    return " ".join([word.lemma_ for word in nlp(text)])



train_df['Title_processed'] = train_df['Title_processed'].apply(Lemmetization)




In [23]:
# train_df.drop(columns='Title_processed', axis=1, inplace=True)

In [24]:
string = train_df.iloc[1, 1]

In [25]:
string.lower()

'carlyle looks toward commercial aerospace (reuters)'

In [26]:
def insert_spaces_at_begining(string):
    string = ' ' + string
    return string

In [ ]:
train_df['Description_preprocessed'] = train_df['Description_preprocessed'].apply(insert_spaces_at_begining)

In [ ]:
train_df['tags'] = train_df['Title_processed'] + train_df['Description_preprocessed']

In [ ]:
train_df.head(1)

In [ ]:
train_df['tags_preprocessed'] = train_df['Title_processed'] + train_df['Description_preprocessed']

In [ ]:
train_df.to_csv('train_transformed.csv', index=False)

In [ ]:
def making_corpus(string):
    data = string.split()
    return data

In [ ]:
string = 'wall st bear claw back black reuterreuter short seller wall street dwindlingband ultra cynic see green'

In [ ]:
making_corpus(string)

In [ ]:
train_df['tags_preprocessed'] = train_df['tags_preprocessed'].apply(making_corpus)

In [ ]:
corpus=[]

In [ ]:
def corpus_formation_full(ls):
    corpus.append(ls)

In [ ]:
train_df['tags_preprocessed'] = train_df['tags_preprocessed'].apply(corpus_formation_full)

In [ ]:
corpus

In [40]:
# from itertools import chain
# flatten_corpus = list(chain.from_iterable(corpus))

In [41]:
#Using pretrained wor2vec

In [42]:
import gensim.downloader

In [ ]:
google_news_word2vec_model = gensim.downloader.load('word2vec-google-news-300')

[===============-----------------------------------] 30.4% 505.3/1662.8MB downloaded

In [ ]:
model = gensim.models.Word2Vec(window=10, min_count=1, workers=8, vector_size=300, sg=1, negative=10)

In [ ]:
model.build_vocab(flatten_corpus)

In [ ]:
model.train(flatten_corpus, total_examples=model.corpus_count, epochs=10)

(28786838, 30699550)

In [ ]:
#Building vocabulary
vocabulary = google_news_word2vec_model.index_to_key

In [ ]:
len(vocabulary)

In [ ]:
word_vec_dict = {}

for i in vocabulary:
      word_vec_dict[i] = google_news_word2vec_model.get_vector(i)
print(len(word_vec_dict))

In [ ]:
tok = Tokenizer()
tok.fit_on_texts(train_df['tags_preprocessed'])
vocab_size = len(tok.word_index) + 1

In [ ]:
def cnt_len_of_each sentence(string):
    length = len(string.split())

In [ ]:
train_df['cnt'] = train_df['tags_preprocessed'].apply(cnt_len_of_each)

In [ ]:
max_len = 

In [ ]:
encd_reviews = tok.texts_to_sequences(train_df['tags_preprocessed'])

In [ ]:
embd_dim = 300

In [ ]:
pad_reviews = pad_sequences(maxlen = max_len, padding='pre', sequences=encd_reviews)

In [ ]:
pad_reviews.shape

In [ ]:
all_embd = np.stack(embd_values)
embd_mean, embd_std = all_embd.mean(), all_embd.std()

In [ ]:
final_embeddings = np.random.normal(embd_mean, embd_std, (vocab_size, embd_dim))
for word , i in tok.word_index.items():
    word_embedding = google_news_word2vec_model.get(word)
    if word_embedding is not None:
        final_embeddings[i] = word_embedding

In [ ]:
import pickle

In [ ]:
final_embeddings = pickle.load(open('final_embeddings_toxic_comments_kaggle.pkl', 'rb'))

In [ ]:
from keras.initializers import Constant
from keras.layers import ReLU, Bidirectional
from keras.layers import Dropout
from keras.layers import GlobalMaxPool1D, BatchNormalization
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping

In [ ]:
model = Sequential()
model.add(Input(shape=(max_len, )))
model.add(Embedding(input_dim = vocab_size,  output_dim = embd_dim, input_length = max_len, weights = [final_embeddings], trainable=False))
model.add(Bidirectional(LSTM(64, return_sequences=True)))
model.add(GlobalMaxPool1D())
# model.add(BatchNormalization())
# model.add(Dense(256, activation='relu'))
# model.add(Dropout(0.2))
# # model.add(BatchNormalization())
# model.add(Dense(128, activation='relu'))
# model.add(Dropout(0.2))
# # model.add(BatchNormalization())
model.add(Dense(64, activation='relu'))
# model.add(BatchNormalization())
# model.add(Dropout(0.2))
model.add(Dense(4, activation='sigmoid'))